In [1]:
# Date: 2025.10.08
# Author: Mihai ANDRIES

## 1. Launch the pyBullet simulator

### 1.1 Import the required libraries

In [1]:
%load_ext autoreload
%autoreload 2
import pybullet as p
import numpy as np
import math
import time
from pandapybullet_minimal.panda_sim import PandaSim
import matplotlib.pyplot as plt
%matplotlib inline
import os.path as path # for loading objects into the scene

pybullet build time: Dec  1 2021 18:33:43


### 1.2 Instantiate a simulation with the Panda robotic arm on top of a table.

In [2]:
panda_sim = PandaSim()

Scene setup
Gravity set to: (0, 0, -10.0)
Table Id: 1
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) UHD Graphics (CML GT2)
GL_VERSION=4.6 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.2.6
Vendor = Intel
Renderer = Mesa Intel(R) UHD Graphics (CML GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on L

### 1.3 Define some useful functions for later use.
- spawnObject(...)
- removeAllSpawnedObjects()
- printObjectPose(objectId)
- printSpawnedObjectsIds()

In [3]:
objectsIdsSpawned = []

def spawnObject(
        objectPath, 
        objectPosition=[0,0,0], 
        objectOrientation=p.getQuaternionFromEuler([0.0, 0.0, 0.0]), 
        scaling=1.0):
    print("Trying to spawn at {} with orientation {}".format(objectPosition, objectOrientation));
    print("Loading the URDF file...");
    # Load (spawn) the object into the scene
    objectIdSpawned = p.loadURDF(fileName=objectPath,
                        basePosition=objectPosition,
                        baseOrientation=objectOrientation,
                        useMaximalCoordinates=0, # bug: https://github.com/bulletphysics/bullet3/issues/4128
                        useFixedBase=0,
                        globalScaling = scaling
                        );
    print("Object created with id: {}".format(objectIdSpawned));
    printObjectPose(objectIdSpawned);
    global objectsIdsSpawned;
    objectsIdsSpawned.append(objectIdSpawned);
    return objectIdSpawned;

# Displays information about the objects in the scene
def printSpawnedObjectsIds():
    numBodiesInScene = p.getNumBodies()
    print("Bodies in the scene: {}".format(numBodiesInScene));
    global objectsIdsSpawned;
    for objectId in objectsIdsSpawned:
        print("\nBody id: {}".format(objectId));
        try:
            bodyInfo = p.getBodyInfo(objectId);
            print("Body Info: {}".format(bodyInfo));
            printObjectPose(objectId);
        except:
            print("An exception occured");
    return
        
# Removes an object from simulation and 
# updates the list of spawned objects 
# that still exist in simulation
def removeObject(objectId):
    # Remove the object from simulation
    p.removeBody(objectId);
    # Remove the object from our list of spawned objects.
    global objectsIdsSpawned;
    objectsIdsSpawned.remove(objectId);
    return 
    
# Removes all spawned objects in the scene 
# (except for the floor plane, the table, and the robot.)
def removeAllSpawnedObjects():
    global objectsIdsSpawned;
    for objectId in objectsIdsSpawned:
        removeObject(objectId);
        # p.removeBody(objectId);
    return;
        
def printObjectPose(objectId):
    # Get position and orientation
    objectPosition, objectOrientationQuaternion = p.getBasePositionAndOrientation(objectId);
    # Print object position and orientation
    print("Object position: \n{}".format(objectPosition));
    # print("Object orientation Quaternion: \n{}".format(objectOrientationQuaternion));
    print("Object orientation Euler: \n{}".format(p.getEulerFromQuaternion(objectOrientationQuaternion)));
    return 

## 2. How to interact with the robot? How to change  the robot state?

### 2.1 Define some time constants for the simulation

In [4]:
sim_sleep_time = 1./240 #By default the simulation step of pyBullet is 1./240
simulation_steps = 480 #Corresponds to 2 seconds

In [5]:
# Function used to progress time in the simulation by n seconds.
def sleepNSeconds(n=1.0):
    simulation_steps = int(n / sim_sleep_time);
    # print(seconds);
    for i in range(simulation_steps):
        time.sleep(sim_sleep_time)
        p.stepSimulation()
        
def resetArmToTableViewingPosition():
    # Reset the arm in the table-viewing position
    simulation_steps = 480
    sim_sleep_time = 1./240 #By default the simulation step of pyBullet is 1./240

    for i in range(simulation_steps):

        controlPositions = [-0.2, 0.0, 0.0, -math.pi / 2.0, 0.0, math.pi / 2.0, math.pi / 4.0]
        panda_sim.send_position(controlPositions)

        time.sleep(sim_sleep_time)

In [6]:
# Warning: do not use during simulation as it overrides physics
# panda_sim.set_q((-0.2, 0.0, 0.0, -math.pi / 2.0, 0.0, math.pi / 2.0, math.pi / 4.0))

### 2.2 Control the robot

#### 2.2.a Velocity control
Uncomment the code to run it.

In [7]:
# for i in range(simulation_steps):

#     controlVelocities = [.1]*7
#     panda_sim.send_velocity(controlVelocities)
         
#     time.sleep(sim_sleep_time)

#### 2.2.b Position control
Uncomment the code to run it.

In [8]:
# simulation_steps = 480
# sim_sleep_time = 1./240 #By default the simulation step of pyBullet is 1./240

# for i in range(simulation_steps):

#     controlPositions = [-0.2, 0.0, 0.0, -math.pi / 2.0, 0.0, math.pi / 2.0, math.pi / 4.0]
#     panda_sim.send_position(controlPositions)
      
#     time.sleep(sim_sleep_time)

#### (!) Tip: Camera movement is possible now

### 2.3 How to run the simulation faster than real-time?
You can run the simulation faster than realtime (if you need to do RL for example).   
(Uncomment the code to run it)

In [9]:
# start = time.time()

# # Velocity control example (start)
# for i in range(simulation_steps):

#     controlVelocities = [.1]*7
#     panda_sim.send_velocity(controlVelocities)
# # Velocity control example (end)    
    
# elapsed = time.time()-start
# print('We just simulated {:.1f} seconds of robot time in {:.1f} seconds'.format(simulation_steps/240, elapsed))

#### 2.3.2 Reset the arm in the table-viewing position

In [10]:
# Reset the arm in the table-viewing position
simulation_steps = 480
sim_sleep_time = 1./240 #By default the simulation step of pyBullet is 1./240

for i in range(simulation_steps):

    controlPositions = [-0.2, 0.0, 0.0, -math.pi / 2.0, 0.0, math.pi / 2.0, math.pi / 4.0]
    panda_sim.send_position(controlPositions)
      
    time.sleep(sim_sleep_time)

### 2.4 How to use the camera on the end-effector?

#### 2.4.1 Display the images seen by the camera inside the simulator (rgb, depth, segmentation masks)

In [11]:
# Acquire data from camera
rgb, depth, seg = panda_sim.grab_eye_in_hand_camera_frame()

#### 2.4.2 Display below the images seen by the camera (rgb, depth, segmentation)

In [12]:
fig, axes = plt.subplots(1, 3)
axes[0].imshow(rgb)
axes[1].imshow(depth)
axes[2].imshow(seg)

#### 2.4.3 How fast is the camera? How many frames can we get per second? How much time is required per frame?
Warning: this function is a bit slow
(Uncomment the code to run it)

In [13]:
# %timeit panda_sim.grab_eye_in_hand_camera_frame()

### 2.5 How to get the robot state?

You can get joint values, joint velocities and jacobian with this function:

In [14]:
q, dq, J = panda_sim.get_qdq_J()

In [15]:
#### 2.5.1 What is the dimension of the Jacobian matrix?

In [16]:
print(J.shape)

(6, 7)


The position of the end-effector of the robot can be queried:

In [17]:
panda_sim.get_x()

array([ 0.54341489, -0.11026741,  1.5044322 ])

### 2.6 Other possibilities for robot control
- Torque control is possible with panda_sim.send_torque('tuple of size 7')
- Cartesian control with fixed orientation is also available with panda_sim.send_cartesian_velocity('tuple of size 3')
- Realtime simulation is possible (panda_sim = PandaSim(realtime=True)). In realtime you can move the robot with the mouse, which can be useful

And of course all of the functionalities of pyBullet!

See https://pythonhosted.org/pybullet/ for the documentation

An example of how to use a pyBullet function:

In [18]:
print(panda_sim);

In [19]:
pos, quat = p.getBasePositionAndOrientation(panda_sim.robot_id);
print('Position of the base of the robot:{}'.format(pos));
print('Orientation of the base of the robot (quaternion):{}'.format(quat));
print('Orientation of the base of the robot (Euler):{}'.format(p.getEulerFromQuaternion(quat)));

Position of the base of the robot:(0.0, 0.0, 0.88)
Orientation of the base of the robot (quaternion):(0.0, 0.0, 0.0, 1.0)
Orientation of the base of the robot (Euler):(0.0, -0.0, 0.0)


# 3. Adding objects into the scene

When importing objects using their relative paths, it is useful to know your current working directory:

In [20]:
import os
print(os.getcwd());

/home/mihai/Documents/IMT Atlantique/Teaching/Master SIIA/UE IML - Robotics (2023-2025)/7. Grasping (2024.03)/Grasping TP/TP_panda_pybullet_grasping_with_solution (2024)/pandapybullet_minimal_TP


### 3.1 Compute where to spawn the objects (we want them on the table)
Identify the location (x,y,z) where you want to spawn the object to grasp.
It should be on top of the table, in the middle, visible to the camera.

Use the PyBullet documentation to identify the appopriate function to use.   
Hint 1: see getCollisionShapeData(...)   
Hint 2: The table has id=1. You will need this information to call the function from Hint 1.

In [21]:
# Fill in the code here.



### 3.2 Load the object into the scene

Spawn the object to grasp. 
Use the right position and orientation for your object.
   
Hint 3: You may use the function "spawnObject" defined in one of the cells above.

In [31]:
objectPath = "./objectsToGrasp/shapegen/cube.urdf";
# Use scaling factor 0.04 when spawning this object.

In [32]:
# Fill in the code here to spawn this object into the scene.



### 3.3 Print list of objects in scene

In [33]:
printSpawnedObjectsIds();    
# p.syncBodyInfo();

Bodies in the scene: 3


### 3.4 Let the object drop onto the table

Hint 5: Use the function "sleepNSeconds".

In [34]:
# Fill in the code here.

### 3.5 Remove object from scene (if required)
Uncomment the code to run it.

In [35]:
# print("Removing object with id: {}".format(objectToGraspId));
# removeObject(objectToGraspId);
# # removeAllSpawnedObjects();
# print("Object removed!");

Error in behaviour: removes a part (link) of the robot, not the object.
Why? It is related to the spawning of objects
https://github.com/bulletphysics/bullet3/issues/4128

# 4. How to grasp the spawned object?
Let us try to grasp two different objects: a cube, and an object with a more complex shape.

### 4.1 Get the position of the object to grasp
You have several options here:
1. Simple option: cheat and use the object position as given by the simulator. See the definition of the spawnObject function above, and the "loadURDF" function it uses.  
Hint 6: you may use the "getBasePositionAndOrientation(...)" function.
2. Difficult option: use the data from your camera to identify the object location. You may cheat and use the segmentation-mask image given by the simnulator.
3. Expert option: use the data from you camera to identify the object location. 

In [36]:
## Compute which point to reach using the arm.
# Fill in the code here




### 4.2 Compute pre-grasp location (0.2m above object centroid)
Hint 7: You may want to reach a point slightly (0.2m) above the object with the origin of your end-effector. This should leave enough space for the end-effector to fit between the wrist and the object.  

In [37]:
# Fill in the code here

### 4.3 Calculate the inverse kinematics to reach that pre-grasp point and orientation.
Compute the target position for the 7 actuated joints of your robotic arm.   
See the PyBullet documentation to learn about the required parameters:   
https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#heading=h.9i02ojf4k3ve

Hint 8: Use the calculateInverseKinematics(...) function.   
Hint 9: You may want to reach the object while pointing the gripper downwards.    


In [38]:
### Fill in your code here.




### 4.3 Reach that pre-grasp pose with the arm

In [39]:
### Fill in your code here.



### 4.4 Open gripper fingers wide

In [40]:
max_opening_width=0.15;
opening_width=max_opening_width;
panda_sim.actuate_gripper(opening_width, force=10);

In [41]:
### Fill in your code here.

### 4.5 Computer target location

In [42]:
### Fill in your code here.

### 4.6 Compute inverse kinematics to reach grasp position

In [43]:
### Fill in your code here.

### 4.7 Reach that grasp pose with the arm
Hint 10: Check the examples above on how to control the robotic arm. You may need the "panda_sim.send_position" function.

In [2]:

### Fill in your code here.

#### Set the friction coefficients for objects

In [3]:
p.changeDynamics(panda_sim.robot_id, 10, lateralFriction=1.0)  # Apply friction to the gripper joints
p.changeDynamics(panda_sim.robot_id, 11, lateralFriction=1.0)  # Apply friction to the gripper joints
p.changeDynamics(objectToGraspId, -1, lateralFriction=1.0)  # Apply friction to the object

# Adjust the restitution or stiffness to make the interaction realistic
p.changeDynamics(panda_sim.robot_id, 10, restitution=0.5)
p.changeDynamics(panda_sim.robot_id, 11, restitution=0.5)

NameError: name 'p' is not defined

### 4.8 Squeeze the object (close fingers)

In [45]:
### Fill in your code here.

### 4.9 Lift the arm, to see if it continues holding the object in its grasp.

In [46]:
### Fill in your code here.


### 4.10 Check if the object is above the table, at a constant location (while the arm holds it).

In [47]:
### Fill in your code here.




Convenience function: reset the arm to a table-viewing position

In [48]:
# resetArmToTableViewingPosition()

# Stop the simulation

In [ ]:
p.disconnect()